<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Python - Testing

by [The Carpentries](https://training.galaxyproject.org/hall-of-fame/carpentries/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Donny Vrins](https://training.galaxyproject.org/hall-of-fame/dirowa/), [Bazante Sanders](https://training.galaxyproject.org/hall-of-fame/bazante1/), [The Carpentries](https://training.galaxyproject.org/hall-of-fame/carpentries/), [Avans Hogeschool](https://training.galaxyproject.org/hall-of-fame/avans-atgm/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- Does the code we develop work the way it should do?
- Can we (and others) verify these assertions for themselves?
- To what extent are we confident of the accuracy of results that appear in publications?

**Objectives**

- Explain the reasons why testing is important
- Describe the three main types of tests and what each are used for
- Implement and run unit tests to verify the correct behaviour of program functions

**Time Estimation: 45M**
</div>


<p>Here we will cover the basics of testing, an important part of software development. Testing lets you know that your code is correct in many situations that matter to you.</p>
<blockquote class="comment" style="border: 2px solid #ffecc1; margin: 1em 0.2em">
<div id="comment" class="box-title" aria-label="comment box: "><i class="far fa-comment-dots" aria-hidden="true"></i><span class="visually-hidden"></span> Comment</div>
<p>This tutorial is significantly based on <a href="https://carpentries.org">the Carpentries</a> lesson <a href="https://carpentries-incubator.github.io/python-intermediate-development/">“Intermediate Research Software Development”</a>.</p>
</blockquote>
<p>Being able to demonstrate that a process generates the right results is important in any field of research, whether it’s software generating those results or not. So when writing software we need to ask ourselves some key questions:</p>
<ul>
<li>Does the code we develop work the way it should do?</li>
<li>Can we (and others) verify these assertions for themselves?</li>
<li>Perhaps most importantly, to what extent are we confident of the accuracy of results that appear in publications?</li>
</ul>
<p>If we are unable to demonstrate that our software fulfills these criteria, why would anyone use it? Having well-defined tests for our software are useful for this, but manually testing software can prove an expensive process.</p>
<p>Automation can help, and automation where possible is a good thing - it enables us to define a potentially complex process in a repeatable way that is far less prone to error than manual approaches. Once defined, automation can also save us a lot of effort, particularly in the long run. In this episode we’ll look into techniques of automated testing to improve the predictability of a software change, make development more productive, and help us produce code that works as expected and produces desired results.</p>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div id="agenda" class="box-title" aria-label="agenda box: ">Agenda</div>
<p>In this tutorial, we will cover:</p>
<ol id="markdown-toc">
<li><a href="#what-is-software-testing" id="markdown-toc-what-is-software-testing">What Is Software Testing?</a></li>
<li><a href="#example-codebase" id="markdown-toc-example-codebase">Example Codebase</a></li>
</ol>
</blockquote>
<h2 id="what-is-software-testing">What Is Software Testing?</h2>
<p>For the sake of argument, if each line we write has a 99% chance of being right, then a 70-line program will be wrong more than half the time. We need to do better than that, which means we need to test our software to catch these mistakes.</p>
<p>We can and should extensively test our software manually, and manual testing is well-suited to testing aspects such as graphical user interfaces and reconciling visual outputs against inputs. However, even with a good test plan, manual testing is very time consuming and prone to error. Another style of testing is automated testing, where we write code that tests the functions of our software. Since computers are very good and efficient at automating repetitive tasks, we should take advantage of this wherever possible.</p>
<p>There are three main types of automated tests:</p>
<ul>
<li><strong>Unit tests</strong> are tests for fairly small and specific units of functionality, e.g. determining that a particular function returns output as expected given specific inputs.</li>
<li><strong>Functional or integration tests</strong> work at a higher level, and test functional paths through your code, e.g. given some specific inputs, a set of interconnected functions across a number of modules (or the entire code) produce the expected result. These are particularly useful for exposing faults in how functional units interact.</li>
<li><strong>Regression tests</strong> make sure that your program’s output hasn’t changed, for example after making changes your code to add new functionality or fix a bug.</li>
<li><strong>Property tests</strong> are an advanced testing strategy to find corner cases.</li>
</ul>
<p>For the purposes of this course, we’ll focus on unit tests. But the principles and practices we’ll talk about can be built on and applied to the other types of tests too.</p>
<h2 id="example-codebase">Example Codebase</h2>


In [ ]:
def rle_encode(input_string):
    count = 1
    prev = ""
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                entry = (prev, count)
                lst.append(entry)
            count = 1
            prev = character
        else:
            count += 1
    entry = (character, count)
    lst.append(entry)
    return "".join([f"{k}{v}" for k, v in lst])


def rle_decode(lst):
    q = ""
    for character, count in zip(lst[::2], lst[1::2]):
        q += character * int(count)
    return q

<p>This is a simple run length encoding and decoding function. It does not handle a lot of corner cases or odd input, but it may do what we need: compressing long strings of repetitive texts. It could potentially provide good genomic data compression. (Spoilers, it generally doesn’t. There are better compression algorithms.)</p>


In [ ]:
print(rle_encode("aaabbc"))

<p>Let’s look at how we can test this cod.</p>
<h2 id="writing-tests-to-verify-correct-behaviour">Writing Tests to Verify Correct Behaviour</h2>
<h3 id="one-way-to-do-it">One Way to Do It?</h3>
<p>One way to test our functions would be to write a series of checks or tests, each executing a function we want to test with known inputs against known valid results, and throw an error if we encounter a result that is incorrect</p>


In [ ]:
test_input = "abba"
test_output = "a1b2a1"

assert rle_encode(test_input) == test_output

<p>So we use the assert keyword - part of Python - to test that our calculated result is the same as our expected result. This function explicitly checks that the two values are the same, and throws an AssertionError if they are not.</p>
<p>Let’s write some more test cases:</p>


In [ ]:
test_input = "wwwwssssb"
test_output = "w5s4b1"
assert rle_encode(test_input) == test_output

test_round_trip = "bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"
assert test_round_trip == rle_decode(rle_encode(test_round_trip))

test_input = "baabaa"
test_output = "b1a2b1a2"
assert rle_encode(test_input) == test_output

<p>However, if we were to enter these in this order, we’ll find we get the following after the first test:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">AssertionError                            Traceback (most recent call last)
&lt;ipython-input-30-05489b2ef047&gt; in &lt;module&gt;
      1 test_input = "wwwwssssb"
      2 test_output = "w5s4b1"
----&gt; 3 assert rle_encode(test_input) == test_output

AssertionError:
</code></pre></div></div>
<p>This tells us our function couldn’t handle these inputs.</p>
<p>We could put these tests in a separate script to automate the running of these tests. But a Python script halts at the first failed assertion, so the second and third tests aren’t run at all. It would be more helpful if we could get data from all of our tests every time they’re run, since the more information we have, the faster we’re likely to be able to track down bugs. It would also be helpful to have some kind of summary report: if our set of tests - known as a <strong>test suite</strong> - includes thirty or forty tests (as it well might for a complex function or library that’s widely used), we’d like to know how many passed or failed.</p>
<p>Going back to our failed first test, what was the issue? As it turns out, the test itself was incorrect, and should have read:</p>


In [ ]:
test_input = "wwwwssssb"
test_output = "w4s4b1"
assert rle_encode(test_input) == test_output

<p>Which highlights an important point: as well as making sure our code is returning correct answers, we also need to ensure the tests themselves are also correct. Otherwise, we may go on to fix our code only to return an incorrect result that <em>appears</em> to be correct. So a good rule is to make tests simple enough to understand so we can reason about both the correctness of our tests as well as our code. Otherwise, our tests hold little value.</p>
<h3 id="using-a-testing-framework">Using a Testing Framework</h3>
<p>Keeping these things in mind, here’s a different approach that builds on the ideas we’ve seen so far but uses a <strong>unit testing framework</strong>. In such a framework we define our tests we want to run as functions, and the framework automatically runs each of these functions in turn, summarising the outputs. And unlike our previous approach, it will run every test regardless of any encountered test failures.</p>
<p>Most people don’t enjoy writing tests, so if we want them to actually do it, it must be easy to:</p>
<ul>
<li>Add or change tests,</li>
<li>Understand the tests that have already been written,</li>
<li>Run those tests, and</li>
<li>Understand those tests’ results</li>
</ul>
<p>Test results must also be reliable. If a testing tool says that code is working when it’s not, or reports problems when there actually aren’t any, people will lose faith in it and stop using it.</p>


In [ ]:
from main import rle_encode, rle_decode
import unittest

class TestRunLengthEncoding(unittest.TestCase):
    def test_encode(self):
      i = "aaabbcccd"
      expected = "a3b2c3d1"
      actual = rle_encode(i)
      self.assertEqual(expected, actual)

    def test_decode(self):
        i = "a3b2c3d1"
        expected = "aaabbcccd"
        actual = rle_decode(i)
        self.assertEqual(expected, actual)

    def test_empty(self):
        test_input = "wwwwssssb"
        test_output = "w4s4b1"
        self.assertEqual(test_input, test_output)

    def test_roundtrip_short(self):
        test_input = "bbb"
        test_output = rle_decode(rle_encode(test_input))
        self.assertEqual(test_input, test_output)

    def test_roundtrip_long(self):
        test_input = "bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"
        test_output = rle_decode(rle_encode(test_input))
        self.assertEqual(test_input, test_output)

if __name__ == '__main__':
    unittest.main()

<p>So here, although we have specified our tests as separate functions in a class, they run the same assertions. Each of these test functions, in a general sense, are called <strong>test cases</strong> - these are a specification of:</p>
<ul>
<li>Inputs</li>
<li>Execution conditions - any imports we might need to do</li>
<li>Testing procedure</li>
<li>Expected outputs</li>
</ul>
<p>And here, we’re defining each of these things for a test case we can run independently that requires no manual intervention.</p>
<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div id="hands-on-run-this-locally" class="box-title" aria-label="hands-on box: Run this locally"><i class="fas fa-pencil-alt" aria-hidden="true"></i><span class="visually-hidden"></span> Hands-on: Run this locally</div>
<ol>
<li>Create a new folder somewhere</li>
<li>Save the RLE encode/decode functions as <code style="color: inherit">main.py</code> in that folder</li>
<li>Save the unittests above as <code style="color: inherit">test.py</code> in that folder</li>
<li>Run <code style="color: inherit">python test.py</code></li>
</ol>
</blockquote>
<p>Going back to our list of requirements, how easy is it to run these tests? We can do this using a Python package called <code style="color: inherit">unittest</code>. This is a built-in testing framework that allows you to write test cases using Python. You can use it to test things like Python functions, database operations, or even things like service APIs - essentially anything that has inputs and expected outputs.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div id="tip-what-about-unit-testing-in-other-languages" class="box-title"><button type="button" aria-controls="tip-what-about-unit-testing-in-other-languages-contents" aria-expanded="true" aria-label="Toggle tip box: What About Unit Testing in Other Languages?"><i class="far fa-lightbulb" aria-hidden="true"></i><span class="visually-hidden"></span> Tip: What About Unit Testing in Other Languages?<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Other unit testing frameworks exist for Python, including Nose and pyunit, and the approach to unit testing can be translated to other languages as well, e.g. FRUIT for Fortran, JUnit for Java (the original unit testing framework), Catch for C++, etc.</p>
</blockquote>
<h3 id="what-about-testing-for-errors">What About Testing for Errors?</h3>
<p>There are some cases where seeing an error is actually the correct behaviour, and Python allows us to test for exceptions. Add this test in <code style="color: inherit">tests/test_models.py</code>:</p>


In [ ]:
import unittest

class ExpectedFailureCase(unittest.TestCase):

    @unittest.expectedFailure
    def test_bad_input():
        from main import rle_decode

        # invalid input, we expect pairs of symbols and numbers.
        rle_decode("a1b")

<p>Run all your tests as before.</p>
<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div id="tip-why-should-we-test-invalid-input-data" class="box-title"><button type="button" aria-controls="tip-why-should-we-test-invalid-input-data-contents" aria-expanded="true" aria-label="Toggle tip box: Why Should We Test Invalid Input Data?"><i class="far fa-lightbulb" aria-hidden="true"></i><span class="visually-hidden"></span> Tip: Why Should We Test Invalid Input Data?<span role="button" class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Testing the behaviour of inputs, both valid and invalid, is a really good idea and is known as <em>data validation</em>. Even if you are developing command line software that cannot be exploited by malicious data entry, testing behaviour against invalid inputs prevents generation of erroneous results that could lead to serious misinterpretation (as well as saving time and compute cycles which may be expensive for longer-running applications). It is generally best not to assume your user’s inputs will always be rational.</p>
</blockquote>
<h3 id="testing-frameworks">Testing Frameworks</h3>
<p>There are many, many testing frameworks for Python that solve similar problems. Look around and see what fits well with your project!</p>
<ul>
<li><a href="https://docs.python.org/3/library/unittest.html">unittest</a> is built into Python’s stdlib.</li>
<li><a href="https://docs.pytest.org/en/7.1.x/">pytest</a> is a heavily recommended framework.</li>
<li><a href="https://nose.readthedocs.io/en/latest/">nose</a> is an alternative to pytest that does many of the same things.</li>
<li><a href="https://hypothesis.readthedocs.io/en/latest/index.html">hypothesis</a> does property testing, the author’s favourite.</li>
<li><a href="https://docs.python.org/3/library/doctest.html">doctest</a> tests examples written directly in the documentation, which keeps test cases close to functions.</li>
</ul>


# Key Points

- The three main types of automated tests are **unit tests**, **functional tests** and **regression tests**.
- We can write unit tests to verify that functions generate expected output given a set of specific inputs.
- It should be easy to add or change tests, understand and run them, and understand their results.
- We can use a unit testing framework like `unittest` to structure and simplify the writing of tests.
- We should test for expected errors in our code.
- Testing program behaviour against both valid and invalid inputs is important and is known as **data validation**.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/python-testing/tutorial.html#feedback) and check there for further resources!
